In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from jax.config import config
config.update('jax_disable_jit', True)

from jax import random as rnd, numpy as jnp
from functools import partial
import jax
from flax.training.train_state import TrainState
import optax

from pyfig import Pyfig

# Todo
- Steps and general JAX implementation details
- pmap across devices
- Run train loop 
- Submit to cluster
- Live results analysis
- gist demo

In [6]:
c = Pyfig(wandb_mode='disabled')
# pprint(c.d)
rng = rnd.PRNGKey(c.seed)
x = c.data.init_walker(rng, n_b=4)
# from hwat import init_walker
# x = init_walker(rng, c.data.n_b, c.data.n_u, c.data.n_d, center=c.data.a, std=0.1)

2022-11-29 15:00:54.832153: W external/org_tensorflow/tensorflow/compiler/xla/service/gpu/nvptx_compiler.cc:497] The NVIDIA driver's CUDA version is 11.4 which is older than the ptxas CUDA version (11.6.55). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [7]:
from hwat import FermiNet
model = c.partial(FermiNet)
vars = model.init(rng, x)  # {'params':p, ... other variables if they exist}
model.apply(vars, x)       # potentially the only way to run something in jax

DeviceArray([ -8.076733, -13.017015, -11.022902, -12.247166], dtype=float32)

In [8]:
def create_train_state(rng):
    x = c.data.init_walker(rng, n_b=4)
    model = c.partial(FermiNet)
    params = model.init(rng, x)
    tx = optax.sgd(c.opt.lr)
    return TrainState.create(
        apply_fn=model.apply, 
        params=params, 
        tx=tx
    )

state = create_train_state(rng)
print([v for v in dir(state) if not v.startswith('_')])

['apply_fn', 'apply_gradients', 'create', 'opt_state', 'params', 'replace', 'step', 'tx']


In [9]:
from hwat import sample

deltar = jnp.array([0.02])
x, v_b = sample(rng, state, x, deltar)
print(v_b.keys())

dict_keys(['deltar', 'acc'])


In [10]:
from hwat import PotentialEnergy

compute_pe = partial(PotentialEnergy(a=c.data.a, a_z=c.data.a_z).apply, {})
print(compute_pe(x))

[ -74.084564  -66.35562  -141.6063   -145.27791 ]


In [32]:
from hwat import compute_ke_b
    
compute_ke_b(state, x)

ValueError: too many values to unpack (expected 3)